# Lecture 4

### ULMfit for fine tuning (RNNs)
- Wikitext103 : language model for next word predication (training from scracth)
- IMDB : language model (took pretrained model from wikipedia model)
- IMDB : sentiment classifier for reviews (fine tuned the language model of IMDB)


### US Patent Phase to Phase Competition
- use "include = object" in dataframe describe function for text columns
- Hugging face model hub : pretrained models (specific to tasks as well)
- Transformers : bit challenging with long documents as it processes in one go (documnets < 2000 words else need GPUs with large memory)
- ULMfit : process gradually by dividing into pieces (documnets > 2000 words)
- Data split : training, validation and test
- outliers : no need to remove all the time

### Transformer library from hugging face
- using classification model and passing continuous variable as lable : does it detect automatically?

# Book Chapter 10

- in general in NLP the pretrained model is trained on a different task.
- language model is a model that has been trained to guess what the next word in a text is (having read the ones before). This kind of task is called self-supervised learning

# Steps to build the model - 

## Tokenizer
- Spacy : word tokenizer
- subword tokenizer : fast ai
- character base : use fast ai one with small vocab size  :)

## Numericalization
- assigning a number to each token (general form of word) in vocabulary 

## Batching
- mini batch with same sequence length
- shuffle the documents not the text inside it

## fine tuning 
- using pretrained model on wiki data 

# coding tips
- summary method in datablock for debugging
- save the intermediate models 

# doubts
- when we are concatenating reviews of different movies and preparing data for language model training : end of one movie review and start of other will become a pair (independent , label) but that doesn't happen when we write.(does xxbos will make model identify that?)
- what is drop_mult=0.3 in learner 
- what is temperature in learn.predict
- how N_WORDS in learn.predict is working 
- grand parent splitter in datablock

# things to explore
- class methods
- data augmentation in NLP
- collecting similar size images in one batch for vision problems 
- Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?


## TODO
- [ ] hugging face implementation
- [x] IMDB implementation

# IMDB Classifier

In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from fastai.text.all import *

## import and read data

In [3]:
path = untar_data(URLs.IMDB)

In [4]:
path.ls()

(#7) [Path('/home/nehaiitr/.fastai/data/imdb/unsup'),Path('/home/nehaiitr/.fastai/data/imdb/train'),Path('/home/nehaiitr/.fastai/data/imdb/tmp_lm'),Path('/home/nehaiitr/.fastai/data/imdb/README'),Path('/home/nehaiitr/.fastai/data/imdb/test'),Path('/home/nehaiitr/.fastai/data/imdb/tmp_clas'),Path('/home/nehaiitr/.fastai/data/imdb/imdb.vocab')]

In [5]:
files = get_text_files(path, folders =['train','test','unsup'])

In [6]:
txt = files[0].open().read()

In [7]:
txt

"Astro Boy<br /><br />One of the few advantages of having a robot child, as opposed to a human one, is that it can process the concept of divorce faster and with less emotional scarring.<br /><br />Fortunately, the little boy in this computer-animated adventure doesn't have to worry about parental separation, because he's dead.<br /><br />When his son Toby (Freddy Highmore) is accidentally killed by a military robot powered by malevolent red energy, Dr. Tenma (Nicholas Cage) re-creates him in a robot form, powered by benevolent blue energy.<br /><br />But when the military becomes interested in his circuitry, Toby must hideout in the old city, where robots participant in gladiatorial games.<br /><br />Based on the Japanese anime, Astro Boy falls far short of its Asian influence: the character designs are flawed, the voice acting is patchy and the story is unexciting.<br /><br />What's more, uptight North American movie makers didn't even bother selling Astro Boy's used underwear in ven

## tokenization

### word tokenizers

In [8]:
# spacy tokenizer
spacy = WordTokenizer()
tokns = first(spacy([txt]))

In [9]:
tokns

(#187) ['Astro','Boy','<','br','/><br','/>One','of','the','few','advantages'...]

In [10]:
# fastai tokenizer on top of spacy
tkn = Tokenizer(spacy)

In [11]:
coll_repr(tkn(txt),10)

"(#214) ['xxbos','xxmaj','astro','xxmaj','boy','\\n\\n','xxmaj','one','of','the'...]"

### subword tokenizers

In [12]:
txts = L(o.open().read() for o in files[:2000])

In [13]:
# size 100 is throwing erorr that vocab size is too small ?
def subword(sz):
    sp = SubwordTokenizer(vocab_sz = sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

In [14]:
subword(3000)

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tmp/texts.out --vocab_size=3000 --model_prefix=tmp/spm --character_coverage=0.99999 --model_type=unigram --unk_id=9 --pad_id=-1 --bos_id=-1 --eos_id=-1 --minloglevel=2 --user_defined_symbols=▁xxunk,▁xxpad,▁xxbos,▁xxeos,▁xxfld,▁xxrep,▁xxwrep,▁xxup,▁xxmaj --hard_vocab_limit=false


'▁As t ro ▁Boy < br ▁/> < br ▁/> O ne ▁of ▁the ▁few ▁advantage s ▁of ▁having ▁a ▁robot ▁child , ▁as ▁oppos ed ▁to ▁a ▁human ▁one , ▁is ▁that ▁it ▁can ▁process ▁the ▁concept ▁of ▁divorce'

In [15]:
subword(200)

'▁A s t ro ▁B o y < br ▁/> < br ▁/> O ne ▁of ▁the ▁f e w ▁a d v an t a g es ▁of ▁ ha v ing ▁a ▁ ro b o t ▁'

## Numericalization

In [52]:
coll_repr(tkn(txt),20)

"(#214) ['xxbos','xxmaj','astro','xxmaj','boy','\\n\\n','xxmaj','one','of','the','few','advantages','of','having','a','robot','child',',','as','opposed'...]"

In [55]:
tokns200 = txts[:200].map(tkn)
tokns200[0]

(#214) ['xxbos','xxmaj','astro','xxmaj','boy','\n\n','xxmaj','one','of','the'...]

In [62]:
num = Numericalize()
num.setup(tokns200)
coll_repr(num.vocab,20)

"(#2008) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the',',','.','a','and','of','to','is','it','in','i'...]"

In [69]:
" ".join([num.vocab[i] for i in num(tkn(txt))[:100]])

"xxbos xxmaj astro xxmaj boy \n\n xxmaj one of the few xxunk of having a robot child , as xxunk to a human one , is that it can process the concept of divorce xxunk and with less emotional xxunk . \n\n xxmaj xxunk , the little boy in this computer - animated adventure does n't have to xxunk about xxunk xxunk , because he 's dead . \n\n xxmaj when his son xxmaj xxunk ( xxunk xxmaj xxunk ) is xxunk killed by a military robot xxunk by xxunk red xxunk , xxmaj dr . xxmaj xxunk ( xxunk"

## Batch creation

### manual version

In [80]:
stream = txts[2]
tokens = tkn(stream)

In [84]:
bs, seq_len = 6, 40
# batches_ls = [ tokens[i:i+seq_len]  for i in range(0,len(tokens),40)]
batches_ls = [ tokens[i*seq_len:(i+1)*seq_len]  for i in range(bs)]

In [85]:
pd.DataFrame(batches_ls)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,xxbos,xxmaj,on,my,first,view,of,xxmaj,little,xxmaj,britain,it,did,n't,strike,me,as,something,that,was,hilariously,funny,.,i,found,it,witty,and,somewhat,entertaining,but,would,n't,have,rated,it,anywhere,near,xxmaj,english
1,sitcoms,such,as,xxmaj,the,xxmaj,office,and,the,like,..,\n\n,xxmaj,however,as,i,watched,it,more,and,began,to,understand,the,characters,and,relate,them,to,stereotypes,of,everyday,people,the,show,began,to,"""",wow,""""
2,me,with,how,many,layers,of,comedy,are,actually,present,.,\n\n,xxmaj,from,xxmaj,daffyth,-,xxmaj,the,only,gay,in,the,village,",",a,stereotype,that,we,know,all,too,well,to,the,xxmaj,cross,xxmaj,dressing,lady
3,boys,who,act,as,though,they,want,to,remain,subtle,in,their,acts,cross,dressing,however,draw,attention,to,themselves,whenever,possible,.,\n\n,xxmaj,these,characters,are,real,.,xxmaj,david,xxmaj,walliams,and,xxmaj,matt,xxmaj,lucas,have
4,really,done,xxmaj,english,comedy,proud,by,making,fictional,takes,on,real,people,.,xxmaj,they,manage,to,capture,all,demographics,of,audience,with,their,mix,of,slapstick,humour,with,more,subtle,comedy,traits,that,audiences,might,not,pick,up
5,on,the,first,viewing,.,\n\n,xxmaj,little,xxmaj,britain,is,a,timeless,classic,-,if,you,do,n't,like,it,at,first,",",give,it,another,go,.,xxmaj,there,is,more,to,it,than,meets,the,eye,…


In [86]:
num200 = tokns200.map(num)

### fastai version

In [87]:
dl = LMDataLoader(num200)

In [88]:
x,y = first(dl)
print(len(x),len(y))

64 64


## Language model training

In [16]:
get_imdb = partial(get_text_files, folders=['train','test','unsup'])

In [17]:
dls_lm = DataBlock(blocks=TextBlock.from_folder(path,is_lm=True),
                  get_items = get_imdb,
                    splitter = RandomSplitter(0.1)
                  ).dataloaders(path, path=path, bs =128, seq_len =80)

In [18]:
dls_lm.show_batch()

,text,text_
0,"xxbos xxmaj do n't be reserved about liking this movie . i love it ! xxmaj it 's one of my favorite movies ever . i hate how comedies can never win awards just for being really really funny . xxmaj the xxmaj oscars that go to comedies are usually won by movies that just are n't hilarious and sometimes have a "" plot "" that just gets in the way . xxmaj anyway , i first saw the movie","xxmaj do n't be reserved about liking this movie . i love it ! xxmaj it 's one of my favorite movies ever . i hate how comedies can never win awards just for being really really funny . xxmaj the xxmaj oscars that go to comedies are usually won by movies that just are n't hilarious and sometimes have a "" plot "" that just gets in the way . xxmaj anyway , i first saw the movie with"
1,"xxup robot xxup holocaust is hardly up ( or down ) to hunt 's level . xxmaj clocking in at a painful 79 minutes ( the box says 90 ) , this is one cheap flick . xxmaj the sets have all the elaborate design of a carnival haunted house and the costumes prove that in the near future everyone will dress like xxmaj john xxmaj travolta in the final dance number of xxup staying xxup alive . xxmaj the","robot xxup holocaust is hardly up ( or down ) to hunt 's level . xxmaj clocking in at a painful 79 minutes ( the box says 90 ) , this is one cheap flick . xxmaj the sets have all the elaborate design of a carnival haunted house and the costumes prove that in the near future everyone will dress like xxmaj john xxmaj travolta in the final dance number of xxup staying xxup alive . xxmaj the atomic"
2,"rarely happens to me . xxmaj the fact that i paid for that does not exactly makes me feel better , as well as the fact the movie deserved the high user rating here . \n\n xxmaj so what is wrong with the movie ? xxmaj it has a fashionable title - "" 44 xxmaj minutes "" . xxmaj one first thinks about "" 15 xxmaj minutes "" , which is by the way a much better movie , but","happens to me . xxmaj the fact that i paid for that does not exactly makes me feel better , as well as the fact the movie deserved the high user rating here . \n\n xxmaj so what is wrong with the movie ? xxmaj it has a fashionable title - "" 44 xxmaj minutes "" . xxmaj one first thinks about "" 15 xxmaj minutes "" , which is by the way a much better movie , but still"
3,"members was trying to provoke the ghost into doing something by calling it names . xxmaj careful doing that , newbie , you might just get possessed . \n\n xxmaj another issue i had is that they rely on technology for xxup everything . xxmaj they 'll set up thousands of dollar 's worth of cameras and other equipment around the place in hopes that they 'll catch something . xxmaj this may be scientifically proved to catch things that","was trying to provoke the ghost into doing something by calling it names . xxmaj careful doing that , newbie , you might just get possessed . \n\n xxmaj another issue i had is that they rely on technology for xxup everything . xxmaj they 'll set up thousands of dollar 's worth of cameras and other equipment around the place in hopes that they 'll catch something . xxmaj this may be scientifically proved to catch things that we"
4,"the sci - fi xxmaj channel - aka xxmaj insomniac xxmaj central - which seems to proudly own the rights to every bad movie ever made . xxmaj if you think their daytime programming is awful , you should check out what they wait until after midnight to air . \n\n xxmaj movies like this . \n\n xxmaj it was a 120 - minute train wreck that i truly could n't take my eyes off of . i have seriously","sci - fi xxmaj channel - aka xxmaj insomniac xxmaj central - which seems to proudly own the rights to every bad movie ever made . xxmaj if you think their daytime programming is awful , you should check out what they wait until after midnight to air . \n\n xxmaj movies like this . \n\n xxmaj it was a 120 - minute train wreck that i truly could n't take my eyes off of . i have seriously been

In [19]:
## what is fp16 and drop mult
learner = language_model_learner(dls_lm,AWD_LSTM,drop_mult = 0.3,metrics=[accuracy,Perplexity()]).to_fp16()

In [ ]:
learner.fit_one_cycle(1,2e-2)

/home/nehaiitr/mambaforge/envs/jlabenv/lib/python3.11/site-packages/torch/amp/autocast_mode.py:204: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
/home/nehaiitr/mambaforge/envs/jlabenv/lib/python3.11/site-packages/torch/cuda/amp/grad_scaler.py:120: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")


epoch,train_loss,valid_loss,accuracy,perplexity,time


# Remaining at kaggle, my computer can't train this big model.